## Imports

In [ ]:
pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from requests import get
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
from random import randint
import numpy as np, pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Scraping

In [ ]:
# pages = np.arange(1, 100, 50)
# # headers = {'Accept-Language': 'en-US,en;q=0.8'} # If this is not specified, the default language is Chinese
# USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0"
# headers = {"user-agent" : USER_AGENT, 'Accept-Language': 'en-US,en;q=0.8'}

# #initialize empty lists to store the variables scraped
# titles = []

# for page in pages:
    
#     # https://www.imdb.com/search/title/?genres=action&start=51&explore=title_type,genres&ref_=adv_nxt
#     # get request
#     response = get("https://www.imdb.com/search/title?genres=action&" 
#                    + "start=" 
#                    + str(page) 
#                    + "&explore=title_type,genres&ref_=adv_nxt", headers=headers)
     
#     # response = get("https://www.imdb.com/chart/top/?ref_=nv_mv_250", headers=headers)

#     sleep(randint(8,15))
    
#     #throw warning for status codes that are not 200
#     if response.status_code != 200:
#         warn('Request: {}; Status code: {}'.format(requests, response.status_code))

#     #parse the content of current iteration of request
#     page_html = BeautifulSoup(response.text, 'html.parser')
        
#     movie_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
    
#     #extract the 50 movies for that page
#     for container in movie_containers:
#         #title
#         title = container.h3.a.text
#         titles.append(title)

In [ ]:
# response = get("https://www.imdb.com/chart/top/?ref_=nv_mv_250", headers=headers)

# sleep(randint(8,15))
    
# #throw warning for status codes that are not 200
# if response.status_code != 200:
#   warn('Request: {}; Status code: {}'.format(requests, response.status_code))

#     #parse the content of current iteration of request
# page_html = BeautifulSoup(response.text, 'html.parser')
        
# movie_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
    
#     #extract the 50 movies for that page
# for container in movie_containers:

#         #conditional for all with metascore
#   if container.find('div', class_ = 'ratings-metascore') is not None:

#             #title
#     title = container.h3.a.text
#     titles.append(title)

## Function Title Scraping per Genre

In [ ]:
pages = np.arange(1, 100, 50)

# headers = {'Accept-Language': 'en-US,en;q=0.8'} # If this is not specified, the default language is Chinese
USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0"
headers = {"user-agent" : USER_AGENT, 'Accept-Language': 'en-US,en;q=0.8'}

#initialize empty lists to store the variables scraped
titles = []

def title(genre):
  for page in pages:
    
    # https://www.imdb.com/search/title/?genres=action&start=51&explore=title_type,genres&ref_=adv_nxt
    # get request
    url = f"https://www.imdb.com/search/title?genres={genre}&start={page}&explore=title_type,genres&ref_=adv_nxt"
    response = get(url, headers=headers)
     
    # response = get("https://www.imdb.com/chart/top/?ref_=nv_mv_250", headers=headers)

    sleep(randint(8,15))
    
    #throw warning for status codes that are not 200
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    #parse the content of current iteration of request
    page_html = BeautifulSoup(response.text, 'html.parser')
        
    movie_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
    
    #extract the 50 movies for that page
    for container in movie_containers:
        #title
        title = container.h3.a.text
        titles.append(title)


In [ ]:
genre = ['action', 'adventure', 'drama', 'comedy']
for genres in genre:
  title(genres)

In [ ]:
title_df = pd.DataFrame({'movie': titles})
title_df

,movie
0,Ghosted
1,The Mandalorian
2,Citadel
3,The Night Agent
4,Peter Pan & Wendy
...,...
395,Once Upon a Time in Hollywood
396,A League of Their Own
397,Rick and Morty
398,Ash vs Evil Dead


In [ ]:
title_df['movie'] = title_df['movie'].str.replace(r'[^\w\s]+', '')
title_df

<ipython-input-14-234103124b02>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  title_df['movie'] = title_df['movie'].str.replace(r'[^\w\s]+', '')


,movie
0,Ghosted
1,The Mandalorian
2,Citadel
3,The Night Agent
4,Peter Pan Wendy
...,...
395,Once Upon a Time in Hollywood
396,A League of Their Own
397,Rick and Morty
398,Ash vs Evil Dead


## Scraping Content 

In [ ]:
def find_imdb_url(query):
  query = query.replace(' ', '+')
  URL = f"https://google.com/search?q={query}+imdb"
  
  resp = get(URL, headers= headers)
  if resp.status_code == 200:
    soup = BeautifulSoup(resp.content, "html.parser") 
  
  hasil = []
  for g in soup.find_all('div', class_='g'):
    anchors = g.find_all('a')
    if anchors:
      link = anchors[0]['href']
    if "imdb.com" in link:
        hasil.append(link)
  return hasil[0]

In [ ]:
query = 'Nope'
find_imdb_url(query)

In [ ]:
def imdb_scrape(URL):
  # desktop user-agent
  resp = get(URL, headers= headers)

  if resp.status_code == 200:
      soup = BeautifulSoup(resp.content, "html.parser")


  for g in soup.find_all('section', class_ = 'ipc-page-background ipc-page-background--base sc-f9e7f53-0 ifXVtO'):
    movie_genre = []
    storyline = ''
    plot = ''

    try:
      for genres in g.find_all('div', class_ ="ipc-chip-list__scroller"):
        for genre in genres.find_all('span', class_ ="ipc-chip__text"):
          movie_genre.append(genre.text)
            
      # Story line
      for story in g.find_all('div', class_ = "ipc-html-content ipc-html-content--base"):
        storyline = story.find('div').text

      # Plot
      for plots in g.find_all('p', class_ ="sc-5f699a2-3 lopbTB"):
        plot = plots.find('span').text

    except:
      pass

  return movie_genre, storyline, plot

In [ ]:
URL = 'https://www.imdb.com/title/tt10954984/'
imdb_scrape(URL)

(['Horror', 'Mystery', 'Sci-Fi'],
 "The biggest praise I can give is the cinematography and camera work. I really enjoy looking at wide shots of landscapes and the slow or sudden movements of the camera pushing the tension and anxiety of the scene without having to rely on dialogue or even any acting.I think the film will be divisive among its audience though. I think this is due to a couple of reasons. First, the story progression feels confusing at times to what we are used to seeing in a thriller or horror. I think labelling it as horror will automatically steer away some people while those that enjoys horror may be disappointed that it lacks enough scares to be called a horror. The trailer I feel also gives away too much and puts the film in a box that might detract viewers. Ultimately, you really have to be open-minded going to this movie. In respect to the genre, Nope really isn't a horror, but more of a blend of some sci-fi, some mystery, some comedy, and luckily, no romance.Non

In [ ]:
all_result = []
for ind, title in enumerate(title_df['movie']):
  print('-----------')
  print(ind, title)
  url = find_imdb_url(title)
  if url is not None:
    value = imdb_scrape(url)
    all_result.append(value)
  else:
    all_result.append(None)

-----------
0 Ghosted
-----------
1 The Mandalorian
-----------
2 Citadel
-----------
3 The Night Agent
-----------
4 Peter Pan  Wendy
-----------
5 The Hunger Games The Ballad of Songbirds and Snakes
-----------
6 John Wick Chapter 4
-----------
7 Barry
-----------
8 The Flash
-----------
9 Star Trek Picard
-----------
10 The Covenant
-----------
11 The Last of Us
-----------
12 Guardians of the Galaxy Vol 3
-----------
13 Dungeons  Dragons Honor Among Thieves
-----------
14 Game of Thrones
-----------
15 The Witcher
-----------
16 The Last Kingdom
-----------
17 AntMan and the Wasp Quantumania
-----------
18 The Rookie
-----------
19 The Last Kingdom Seven Kings Must Die
-----------
20 Rabbit Hole
-----------
21 Transformers Rise of the Beasts
-----------
22 Sweet Tooth
-----------
23 Sisu
-----------
24 Avatar The Way of Water
-----------
25 Mighty Morphin Power Rangers Once  Always
-----------
26 The Flash
-----------
27 Kisi Ka Bhai Kisi Ki Jaan
-----------
28 Demon Slayer Kimetsu

UnboundLocalError: ignored

In [ ]:
final_result = pd.DataFrame(all_result, columns=["Genre", "Storyline", 'plot'])

In [ ]:
final_result

,Genre,Storyline,plot
0,"[Action, Adventure, Comedy]",Seriously. This isn't rocket science I underst...,Cole falls head over heels for enigmatic Sadie...
1,"[Action, Adventure, Fantasy]",This show is such a refreshing breeze of great...,The travels of a lone bounty hunter in the out...
2,"[Action, Drama, Thriller]",Mason Kane (Richard Madden) was once one of th...,"Global spy agency Citadel has fallen, and its ..."
3,"[Action, Drama, Thriller]","Despite the low budget feel to this series, it...",Low-level FBI agent Peter Sutherland works in ...
4,"[Action, Adventure, Comedy]","I went into this neutral, excited that Disney ...",Based on J. M. Barrie's novel Peter and Wendy ...
...,...,...,...
335,"[Comedy, Drama, History]",Babylon had some moments that were better than...,A tale of outsized ambition and outrageous exc...
336,"[Comedy, Romance]",When my boyfriend tuned into the premiere I th...,A woman who moves into an apartment across the...
337,"[Comedy, Drama]","I was on a long flight, and was planning to ch...",Four very different thirtysomething working-mo...
338,"[Animation, Comedy]","South Park is one of the most misunderstood, a...",Follows the misadventures of four irreverent g...


In [ ]:
title_df_new = title_df.iloc[:340]
title_df_new

,movie
0,Ghosted
1,The Mandalorian
2,Citadel
3,The Night Agent
4,Peter Pan Wendy
...,...
335,Babylon
336,The Big Bang Theory
337,Workin Moms
338,South Park


In [ ]:
df = pd.merge(title_df_new, final_result, left_index=True, right_index=True)
df

,movie,Genre,Storyline,plot
0,Ghosted,"[Action, Adventure, Comedy]",Seriously. This isn't rocket science I underst...,Cole falls head over heels for enigmatic Sadie...
1,The Mandalorian,"[Action, Adventure, Fantasy]",This show is such a refreshing breeze of great...,The travels of a lone bounty hunter in the out...
2,Citadel,"[Action, Drama, Thriller]",Mason Kane (Richard Madden) was once one of th...,"Global spy agency Citadel has fallen, and its ..."
3,The Night Agent,"[Action, Drama, Thriller]","Despite the low budget feel to this series, it...",Low-level FBI agent Peter Sutherland works in ...
4,Peter Pan Wendy,"[Action, Adventure, Comedy]","I went into this neutral, excited that Disney ...",Based on J. M. Barrie's novel Peter and Wendy ...
...,...,...,...,...
335,Babylon,"[Comedy, Drama, History]",Babylon had some moments that were better than...,A tale of outsized ambition and outrageous exc...
336,The Big Bang Theory,"[Comedy, Romance]",When my boyfriend tuned into the premiere I th...,A woman who moves into an apartment across the...
337,Workin Moms,"[Comedy, Drama]","I was on a long flight, and was planning to ch...",Four very different thirtysomething working-mo...
338,South Park,"[Animation, Comedy]","South Park is one of the most misunderstood, a...",Follows the misadventures of four irreverent g...
